# Convolutional Neural Network

강아지와 고양이를 분류하는 classifier system을 만든다고 가정한다.  
<img src="https://drive.google.com/uc?export=download&id=1RU8Kr1JDKJEDwfEjiMVY-3fEIVuOE4h7">

그림과 같이 두 가지 module로 나눌 수 있다.
- Feature Extractor : 입력 데이터를 받으 다음, 미리 설계한 알고리즘을 통해 추출한 값.

- 좋은 feature 값은 강아지와 고양이간에는 서로 다른(각격이 큰) 값이며, 같은 클래스내(종류나 색깔이 다르지만 강아지에 속하는 경우)에서는 서로 유사한 값을 가져야 한다.

- Classifier : 구한 feature 값을 기반으로 분류 결과를 구한다.

전반적인 CNN(Convolutional Neural Network)의 구조.
<img src="https://drive.google.com/uc?export=download&id=1XjexJww9ERuPPSGpFhTbmlT0GlcUI5uQ">

In [16]:
## Feature Extractor

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten

batch_size, img_height, img_width, img_channel = 32, 28, 28, 3
conv_filters, conv_kernel_size = 5, 3
pool_size, pool_strides = 2, 2

X = tf.random.normal(shape=(batch_size, img_height, img_width, img_channel))

conv1 = Conv2D(filters=conv_filters,
              kernel_size=conv_kernel_size,
              padding="same",
              activation="relu")
pool1 = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

conv2 = Conv2D(filters=conv_filters,
              kernel_size=conv_kernel_size,
              padding="same",
              activation="relu")
pool2 = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

flatten = Flatten() ## dense layer의 input으로 사용하기 위함.

print(f"Input : {X.shape} \n")
X = conv1(X)
W, B = conv1.get_weights()
print(f"Conv1 W / B : {W.shape}, {B.shape}")
print(f"Conv1 output : {X.shape} \n")
X = pool1(X) 
print(f"Pool1 output : {X.shape} \n")

X = conv2(X)
W, B = conv2.get_weights()
print(f"Conv2 W / B : {W.shape}, {B.shape}")
print(f"Conv2 output : {X.shape} \n")
X = pool2(X)
print(f"Pool2 output : {X.shape} \n")

X = flatten(X)
print(f"Flatten output : {X.shape}")

Input : (32, 28, 28, 3) 

Conv1 W / B : (3, 3, 3, 5), (5,)
Conv1 output : (32, 28, 28, 5) 

Pool1 output : (32, 14, 14, 5) 

Conv2 W / B : (3, 3, 5, 5), (5,)
Conv2 output : (32, 14, 14, 5) 

Pool2 output : (32, 7, 7, 5) 

Flatten output : (32, 245)


In [17]:
## Classifier

from tensorflow.keras.layers import Dense

n_neurons = [50, 25, 10]

dense1 = Dense(units=n_neurons[0], activation="relu")
dense2 = Dense(units=n_neurons[1], activation="relu")
dense3 = Dense(units=n_neurons[2], activation="softmax")

print(f"Input(after flatten) : {X.shape} \n")
X = dense1(X)
W,B = dense1.get_weights()
print(f"Dense1 W/B : {W.shape}, {B.shape}")
print(f"Dense1 output : {X.shape} \n")

X = dense2(X)
W,B = dense2.get_weights()
print(f"Dense2 W/B : {W.shape}, {B.shape}")
print(f"Dense2 output : {X.shape} \n")

predictions = dense3(X)
W,B = dense3.get_weights()
print(f"Dense3 W/B : {W.shape}, {B.shape}")
print(f"Dense3 output : {predictions.shape} \n")

Input(after flatten) : (32, 245) 

Dense1 W/B : (245, 50), (50,)
Dense1 output : (32, 50) 

Dense2 W/B : (50, 25), (25,)
Dense2 output : (32, 25) 

Dense3 W/B : (25, 10), (10,)
Dense3 output : (32, 10) 



In [18]:
## Loss function

from tensorflow.keras.losses import CategoricalCrossentropy

Y = tf.random.uniform(minval=0, maxval=10, shape=(batch_size, ), dtype=tf.int32)
Y = tf.one_hot(Y, depth=10)

loss_object = CategoricalCrossentropy()
loss = loss_object(Y, predictions)
print(f"Loss : {loss.shape}, {loss.numpy()}")

Loss : (), 2.9727468490600586


In [19]:
## Sequential model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

batch_size, img_height, img_width, img_channel = 4, 28, 28, 3
conv_filters = [10, 20, 30]
dense_neurons = [50, 30, 10]
kernel_size = 3
padding = "same"
pool_size, strides = 2, 2

x = tf.random.normal(shape=(batch_size, img_height, img_width, img_channel))

model = Sequential()

model.add(Conv2D(filters=conv_filters[0], kernel_size=kernel_size, padding=padding, activation="relu"))
model.add(MaxPooling2D(pool_size=pool_size, strides=strides))

model.add(Conv2D(filters=conv_filters[1], kernel_size=kernel_size, padding=padding, activation="relu"))
model.add(MaxPooling2D(pool_size=pool_size, strides=strides))

model.add(Conv2D(filters=conv_filters[2], kernel_size=kernel_size, padding=padding, activation="relu"))
model.add(MaxPooling2D(pool_size=pool_size, strides=strides))

model.add(Flatten())

model.add(Dense(units=dense_neurons[0], activation="relu"))
model.add(Dense(units=dense_neurons[1], activation="relu"))
model.add(Dense(units=dense_neurons[2], activation="softmax"))

predictions = model(x)
print(predictions.shape)

(4, 10)


In [20]:
## model subclassing

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

class TestCNN(Model):
    def __init__(self):
        super(TestCNN, self).__init__()

        self.conv1 = Conv2D(filters=conv_filters[0], kernel_size=kernel_size, padding=padding, activation="relu")
        self.pool1 = MaxPooling2D(pool_size=pool_size, strides=strides)

        self.conv2 = Conv2D(filters=conv_filters[1], kernel_size=kernel_size, padding=padding, activation="relu")
        self.pool2 = MaxPooling2D(pool_size=pool_size, strides=strides)

        self.conv3 = Conv2D(filters=conv_filters[2], kernel_size=kernel_size, padding=padding, activation="relu")
        self.pool3 = MaxPooling2D(pool_size=pool_size, strides=strides)

        self.flatten = Flatten()

        self.dense1 = Dense(units=dense_neurons[0], activation="relu")
        self.dense2 = Dense(units=dense_neurons[1], activation="relu")
        self.dense3 = Dense(units=dense_neurons[2], activation="softmax")

    def call(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.pool3(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)

        return x

batch_size, img_height, img_width, img_channel = 4, 28, 28, 3
conv_filters = [10, 20, 30]
dense_neurons = [50, 30, 10]
kernel_size = 3
padding = "same"
pool_size, strides = 2, 2

x = tf.random.normal(shape=(batch_size, img_height, img_width, img_channel))

model = TestCNN()
y = model(x)

In [21]:
## layer subclassing with Sequential

import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

class MyConv(Layer):
    def __init__(self, filters):
        super(MyConv, self).__init__()

        self.conv = Conv2D(filters=filters, kernel_size=kernel_size, padding=padding, activation="relu")
        self.pool = MaxPooling2D(pool_size=pool_size, strides=strides)
        
    def call(self, x):
        x = self.conv(x)
        x = self.pool(x)

        return x

model = Sequential()

model.add(MyConv(conv_filters[0]))
model.add(MyConv(conv_filters[1]))
model.add(MyConv(conv_filters[2]))

model.add(Flatten())

model.add(Dense(units=dense_neurons[0], activation="relu"))
model.add(Dense(units=dense_neurons[1], activation="relu"))
model.add(Dense(units=dense_neurons[2], activation="softmax"))

predictions = model(x)
print(predictions.shape)

(4, 10)


In [22]:
## model & layer subclassing

import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

class MyConv(Layer):
    def __init__(self, filters):
        super(MyConv, self).__init__()

        self.conv = Conv2D(filters=filters, kernel_size=kernel_size, padding=padding, activation="relu")
        self.pool = MaxPooling2D(pool_size=pool_size, strides=strides)
        
    def call(self, x):
        x = self.conv(x)
        x = self.pool(x)

        return x

class TestCNN(Model):
    def __init__(self):
        super(TestCNN, self).__init__()

        self.myconv1 = MyConv(conv_filters[0])
        self.myconv2 = MyConv(conv_filters[1])
        self.myconv3 = MyConv(conv_filters[2])

        self.flatten = Flatten()

        self.dense1 = Dense(units=dense_neurons[0], activation="relu")
        self.dense2 = Dense(units=dense_neurons[1], activation="relu")
        self.dense3 = Dense(units=dense_neurons[2], activation="softmax")

    def call(self, x):
        x = self.myconv1(x)
        x = self.myconv2(x)
        x = self.myconv3(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)

        return x

model = TestCNN()
y = model(x)

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense


class LeNet (Model):
  def __init__(self):
    super(LeNet, self).__init__()

    self.convl = Conv2D(filters=6, kernel_size=5, padding='same',
                        activation='tanh')
    self.convl_pool = AveragePooling2D(pool_size=2, strides=2)
    self.conv2 = Conv2D(filters=16, kernel_size=5, padding='same',
                        activation='tanh')
    self.conv2_pool = AveragePooling2D(pool_size=2, strides=2)
    self.conv3 = Conv2D(filters=120, kernel_size=5, padding= 'same',
                        activation='tanh')
    self.flatten = Flatten()
    self.densel = Dense(units=84, activation='tanh')
    self.dense2 = Dense(units=10, activation='softmax')

  def call(self, x):
    print("x: {}".format(x.shape))
    x = self.convl(x)
    print("x: {}".format(x.shape))
    x = self.convl_pool(x)
    print("x: {}".format(x.shape))
    x = self.conv2(x)
    print("x: {}".format(x.shape))
    print("x: {}".format(x.shape))
    x = self.conv3(x)
    print("x: {}".format(x.shape))
    x = self. flatten(x)
    print("x: {}".format(x.shape))
    x = self.densel(x) 
    print("x: {}".format(x.shape))
    x = self.dense2(x)
    print("x: {}".format(x.shape))
    return x

model = LeNet()

x = tf.random.normal(shape=(32, 28, 28, 1))
predictions = model(x)

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense


class ConvLayer(Layer):
  def __init__(self, filters, padding, pool=True):
    super(ConvLayer, self).__init__()
    self.pool = pool

    self.conv = Conv2D(filters=6, kernel_size=5, padding='same',
                      activation='tanh')
    
    if self.pool == True:
      self.conv_pool = AveragePooling2D(pool_size=2, strides=2)

    def call(self,x):
      x = self.conv(x)
      if self.pool == True:
         x = self.conv_pool(x)
      return x


class LeNet(Model):
  def __init__(self):
      super(LeNet, self).__init__()
      self.convl = ConvLayer(filters=6, padding='same')
      self.conv2 = ConvLayer(filters=16, padding='valid')
      self.conv3 = ConvLayer(filters=120, padding='valid', pool=False)
      self.flatten = Flatten()

      self.dense1 = Dense(units=84, activation='tanh')
      self.dense2 = Dense(units=10, activation='softmax')

  def call(self, x):
      x = self.convl(x)
      x = self.conv2(x)
      x = self.conv3(x)

      x = self.flatten(x)
      x = self.dense1(x)
      x = self.dense2(x)
      return x


model = LeNet()

x = tf.random.normal(shape=(32, 28, 28, 1))
predictions = model(x)

In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

from tensorflow.keras.datasets import mnist
from tensorflow.keras.losses import SparseCategoricalCrossentropy


########LeNet Implementation#########
class ConvLayer(Layer):
  def __init__ (self, filters, padding, pool=True):
    super(ConvLayer, self).__init__()
    self.pool = pool
    self.conv = Conv2D(filters=6, kernel_size=5, padding='same',
                      activation='tanh')
    
    if self.pool == True:
      self.conv_pool = AveragePooling2D(pool_size=2, strides=2)

    def call(self,x):
      x = self.conv(x)
      if self.pool == True:
        x = self.conv_pool(x)
      return x

class LeNet (Model):
  def __init__(self):
      super(LeNet, self).__init__()
      self.convl = ConvLayer(filters=6, padding='same')
      self.conv2 = ConvLayer(filters=16, padding='valid')
      self.conv3 = ConvLayer(filters=120, padding='valid', pool=False)
      self.flatten = Flatten()

      self.dense1 = Dense(units=84, activation='tanh')
      self.dense2 = Dense(units=10, activation ='softmax')

  def call(self, x):
      x = self.convl(x)
      x = self.conv2(x)
      x = self.conv3(x)
      x = self.flatten(x)
      x = self.dense1(x)
      x = self.dense2(x)
      return x


model = LeNet()

x = tf.random.normal(shape=(32, 28, 28, 1))
predictions = model(x)

######Dataset Perparation#####
(train_images, train_labels), _ = mnist. load_data()
train_images = np.expand_dims(train_images, axis=3).astype(np.float32)
train_labels = train_labels.astype(np. int32)     
              
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_ds = train_ds.batch(32)

#####Forward Propagation######
model = LeNet()
loss_object = SparseCategoricalCrossentropy()

for images, labels in train_ds:
    predictions = model(images)
    loss = loss_object(labels, predictions)
    print(loss)
    break